In [14]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (315 kB/s)
Reading package lists... Done


In [15]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-12 00:58:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.42MB/s    in 0.2s    

2023-04-12 00:58:28 (5.42 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [17]:
from pyspark import SparkFiles
# Load in the review data from S3 into the dataframe
url = "https://unbearable-1-project-bucket.s3.us-east-2.amazonaws.com/LLCP2021.csv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+------+------+--------+------+----+-----+--------+----------+------------+--------+--------+--------+--------+--------+-------+-------+--------+-------+------+--------+--------+--------+--------+--------+-------+-------+--------+--------+-------+--------+-------+------+-------+--------+--------+--------+--------+--------+--------+--------+--------+-------+------+--------+-------+--------+--------+--------+--------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+-------+-----+--------+--------+--------+--------+--------+-------+--------+-------+--------+-------+-------+----+-----+------+--------+--------+--------+--------+--------+-------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-------+-------+-------+-------+-------+

In [22]:
new_df = df.select(["_STATE",
                    "DIABETE4",
                    "_BMI5",
                    "SMOKE100",
                    "DRNK3GE5",
                    "EDUCA",
                    "GENHLTH",
                    "_AGEG5YR",
                    "_AGE65YR",
                    "_AGE80",
                    "EXERANY2",
                    "FRUIT2",
                    "VEGETAB2",
                    "INCOME3",
                    "MEDCOST1",
                    "_HISPANC",
                    "_MRACE1",
                    "_PRACE1",
                    "_SEX"])
new_df.show()

+------+--------+------+--------+--------+-----+-------+--------+--------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|_STATE|DIABETE4| _BMI5|SMOKE100|DRNK3GE5|EDUCA|GENHLTH|_AGEG5YR|_AGE65YR|_AGE80|EXERANY2|FRUIT2|VEGETAB2|INCOME3|MEDCOST1|_HISPANC|_MRACE1|_PRACE1|_SEX|
+------+--------+------+--------+--------+-----+-------+--------+--------+------+--------+------+--------+-------+--------+--------+-------+-------+----+
|   1.0|     3.0|1454.0|     1.0|    null|  4.0|    5.0|    11.0|     2.0|  70.0|     2.0| 101.0|   101.0|    5.0|     2.0|     2.0|    1.0|    1.0| 2.0|
|   1.0|     1.0|  null|     2.0|    null|  6.0|    3.0|    10.0|     2.0|  67.0|     1.0| 101.0|   207.0|   77.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|   1.0|     1.0|2829.0|     2.0|    null|  4.0|    2.0|    11.0|     2.0|  72.0|     2.0| 101.0|   203.0|    3.0|     2.0|     2.0|    2.0|    2.0| 2.0|
|   1.0|     1.0|3347.0|     2.0|     1.0|  4.0|    2.0|     9.0|     1.0|  

### Clean up Columns